In [1]:
import numpy as np
import random
import pickle
import nltk
import json
import re
import os
from HMM import unsupervised_HMM
from HMM_helper import sample_sentence, parse_observations
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import cmudict
from sklearn.feature_extraction.text import TfidfVectorizer

Lets first read in the sonnets and tokenize them. We will tokenize them both by line and by poem.  

In [3]:
f = open(os.getcwd()+'/data/shakespeare.txt')
lines = f.readlines()
all_words = []
new_lines = []
sonnets=[]
sonnet=[]
poems = []


for line in lines:
    line = line.strip()
    line = line.lower()
    newPun = "!?.;:'#$%&(,)*+/<=>@][^_`{|}~-"
#     line = ''.join(c for c in line if c not in newPunct)
    for punct in newPun:
        line = line.replace(punct, '')
    if line.isdigit():
        sonnets.append(sonnet)
        sonnet = []
    elif line.strip() == '':
        pass
    else:
        sonnet.append(line)
sonnets.append(sonnet)
del sonnets[0]


In [4]:
line_tokens = [] # tokens per line
poem_tokens = [] # tokens per poem
single_poem_tokens = []
words = []

for sonnet in sonnets:
    poem_tokens.append(single_poem_tokens)
    single_poem_tokens = []
    for line in sonnet:
        line_tokens.append(nltk.word_tokenize(line)) # this is all tokens in each line
        single_poem_tokens += nltk.word_tokenize(line)
        words += nltk.word_tokenize(line)
del(poem_tokens[0])

In [5]:
len(words)

17371

The function below will create a dictionary we will use later to ensure the sonnet rhyme scheme is adhered to.

In [6]:
def make_dictionary(dictionary, word1, word2):
    """
    This function takes in a dictionary and two words. 
    It populates the dictionary with keys: word1
    values: word2
    
    """
    if word1 in dictionary:
        word_list = dictionary.get(word1)
        if word2 not in word_list:
            new_word_list = dictionary[word1]
            new_word_list.append(word2)
            dictionary[word1] = new_word_list
    else:
        dictionary[word1] = [word2]

    if word2 in dictionary:
        word_list = dictionary.get(word2)
        if word1 not in word_list:
            new_word_list = dictionary[word2]
            new_word_list.append(word1)
            dictionary[word2] = new_word_list
    else:
        dictionary[word2] = [word1]

    return dictionary

This lower cell has two functions 1) It reads in the syllable dictionary, then returns the list of words and the number of syllables in either the line or whole poem. 2) It creates a rhyming dictionary with the keys being words found in the sonnets and the values being words that rhyme with them

In [7]:
# syllable_dictionary is a dictionary with all the words in the sonnets
# and how many syllables each word is 
syllables = {}
with open("./data/Syllable_dictionary.txt") as syllableDict:
    for line in syllableDict:
        split = line.split()
        if len(split) == 3:
            (key, end, val) = line.split()
        else:
            (key, val) = line.split()

        # they're ordered by syllable length, so sometimes the E is last
        try:
            syllables[key] = int(val)
        except:
            syllables[key] = int(end)

rhymes = {}
# sonnet format: abab cdcd efef gg
for j in range(len(line_tokens)-1):
    line = line_tokens[j]
    last_word = line[-1]
    i = (j % 14) + 1
    # using our dictionary function to create a dict where the values of a given key are all the words that rhyme with it
    if i == 1 or i == 2 or i == 5 or i == 6 or i == 9 or i == 10:
        rhymes = make_dictionary(rhymes, last_word, line_tokens[j+2][-1])
    elif i == 13:
        rhymes = make_dictionary(rhymes, last_word, line_tokens[j+1][-1])
    


This next cell returns the words in the poems as sequences, with a POS tag to each one and finally a list of features in each sonnet. 

In [8]:
sequences = [] 
seq_encode = [] 
features = []

# look through all tokens and 
for obs in poem_tokens:
    # assigning a POS tag to each token in that line
    seq = pos_tag(obs)
    poemFeatures = []
    # if it's a new sequence, add it to the list
    for pair in seq: 
        if pair[1] not in sequences:
            sequences.append(pair[1])
            seq_encode.append([])
            seq_encode[sequences.index(pair[1])].append([pair[0], 1])
        else: 
            firstCol = [row[0] for row in seq_encode[sequences.index(pair[1])]]
            if pair[0] not in firstCol:
                seq_encode[sequences.index(pair[1])].append([pair[0], 1])
            else:
                index = firstCol.index(pair[0])
                seq_encode[sequences.index(pair[1])][index][1] += 1
        
        # poemFeatures now contains the features of each token in the line
        poemFeatures.append(sequences.index(pair[1]))


    features.append(poemFeatures)


This function generates a word given a specific emission probability and the sequence encodings from earlier. It will be used in the generation of the sonnet below.

In [9]:
def generate_words(emission, seq_encode, syllables, reverse=False, lastWord=None):
    '''
    This function generates a string given the emissions and the probabilities 
    of a word being emitted given a certain
    
    '''
    done = False
    if reverse:
        assert(lastWord is not None)
        while not done:
            emStr = lastWord
            try:
                syllableCount = syllables[lastWord]
            except:
                syllableCount = 2
                print(lastWord)
            for obs in emission:
                emRate = [row[1] for row in seq_encode[obs]]
                emWords = [row[0] for row in seq_encode[obs]]
                emRate = np.array(emRate)
                emRate = emRate/sum(emRate)

                index = np.random.choice(np.arange(len(emRate)), p=emRate)
                newWord = emWords[index]
                try:
                    syllableCount += syllables[newWord]
                except:
                    syllableCount += 2
                    print(newWord)
                emStr = newWord + ' ' + emStr
                if syllableCount == 10:
                    done = True
                    break
    else:
        while not done:
            emStr = ''
            syllableCount = 0
            for obs in emission: 
                emRate = [row[1] for row in seq_encode[obs]]
                emWords = [row[0] for row in seq_encode[obs]]
                emRate = np.array(emRate)
                emRate = emRate/sum(emRate)

                index = np.random.choice(np.arange(len(emRate)), p=emRate)
                newWord = emWords[index]
                syllableCount += syllables[newWord]
                emStr = emStr + newWord + ' '
                if syllableCount == 10:
                    done = True
                    break
    return emStr



#### model 

In [10]:
# Choose our model parameters
num_hidden_states = 25
iterations = 100

# train our model on the features
HMM = unsupervised_HMM(features, num_hidden_states, iterations)

# generate emission probabilities and states
emission, states = HMM.generate_emission(10)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


#### Now lets generate our sonnet

Note that the output of this was submitted on Piazza

In [ ]:
# abab cdcd efef gg
sonnet = ["" for x in range(14)]
line_idx = [0, 1, 4, 5, 8, 9, 12]
for i in line_idx:
    # choose a random word in the dictionary
    key, val = random.choice(list(rhymes.items()))
    # choose a random word that rhymes with the previous one
    pair = np.random.choice(val)
    sonnet[i] += str(key)
    if i < 12:
        sonnet[i+2] += str(pair)
    else:
        sonnet[i+1] += str(pair)
for i in range(len(sonnet)):
    line = generate_words(emission, seq_encode, syllables, True, sonnet[i])
    sonnet[i] = line
sonnet = "\n".join(sonnet)
with open('sonnet1.txt', 'w') as f:
    f.write(str(sonnet))
print(sonnet)



#### Visualization

In [ ]:
A = np.array(HMM.A)
O = np.array(HMM.O)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation


plt.imshow(A, vmax=1.0)
plt.colorbar()
plt.title('Sparsity of A matrix')
plt.show()


In [ ]:
plt.imshow(np.array(O)[:, :O.shape[1]], vmax=0.1, aspect='auto')
plt.colorbar()
plt.title('Sparsity of O matrix')
plt.show()


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


newPunct = "!#$%&()*+,./:';<=>?@[\]^_`{|}~"
words = [x for x in words if x not in newPunct]
data = sorted([(w, words.count(w)) for w in set(words)], key = lambda x:x[1], reverse=True)[:40] 
most_words = [x[0] for x in data]
most_words = sorted(most_words)
del(most_words[0]) # this was an 's'
times_used = [int(x[1]) for x in data]
del(times_used[0])

Plotting the word frequencies!

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(most_words, height=times_used, color = 'grey', edgecolor = 'black',  width=.5)
plt.xticks(rotation=45, fontsize=18)
plt.yticks(rotation=0, fontsize=18)
plt.xlabel('Most Common Words:', fontsize=18)
plt.ylabel('Number of Occurences:', fontsize=18)
plt.title('Most Commonly Used Words: %s' % ('data/shakespeare.txt'), fontsize=24)
plt.show()